In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import Input, Conv1D, GlobalAvgPool2D, GlobalAveragePooling1D, \
    Dropout, Dense, Activation, Concatenate, Multiply, MaxPool2D, Add,  \
    LSTM, Bidirectional, Conv2D, AveragePooling2D, BatchNormalization, Flatten, GlobalAveragePooling2D, \
    GlobalMaxPooling2D, Reshape, Permute, multiply, Lambda, add, subtract, MaxPooling2D, GRU, ReLU, MaxPooling1D 
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import roc_auc_score, roc_curve
from keras.models import Model, load_model
from keras import backend as K
from sklearn.model_selection import KFold
from keras.layers import Embedding
import matplotlib as mpl
import matplotlib.pyplot as plt
from keras.regularizers import l1, l2
from keras.utils import to_categorical
from keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from Bio import SeqIO
mpl.use('TkAgg')
import warnings
warnings.filterwarnings("ignore")
import math
from tensorflow.keras import layers
from tensorflow import keras
from keras import initializers

2024-06-12 10:20:44.293295: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-12 10:20:44.709992: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-12 10:20:45.467873: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # 不全部占满显存，按需分配当allow_growth设置为True时，分配器将不会指定所有的GPU内存，而是根据需求增长
session = tf.compat.v1.Session(config=config)

2024-06-12 10:20:51.638615: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11123 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6


In [3]:
def read_fasta(file_path):
    '''File_path: Path to the fasta file
       Returns: List of sequence
    '''
    one=list(SeqIO.parse(file_path,'fasta'))
    return one

In [4]:
def protein_to_Kmer(seqs,K):
    numeric_sequences = []
    base_to_index = {'A': 0, 'C': 1, 'D': 2, 'E': 3,
                     'F': 4, 'G': 5, 'H': 6, 'I': 7, 'K': 8, 'L': 9, 'M': 10,
                     'N': 11, 'P': 12, 'Q': 13, 'R': 14, 'S': 15, 'T': 16, 
                     'V': 17, 'W': 18, 'Y': 19,'X': 20
                     }
    for dna_sequence in seqs:
        numeric_sequence = []
        for base in dna_sequence:
            numeric_sequence.append(base_to_index[base])

        kmer_sequence = []
        for i in range(len(numeric_sequence) - (K - 1)):
            kmer = numeric_sequence[i:i + K]
            kmer_marge = [int("".join(map(str, kmer)))]
            kmer_sequence.append(kmer_marge)

        numeric_sequences.append(kmer_sequence)
    return numeric_sequences

In [5]:
# Read the training set
    
train_pos_seqs = np.array(read_fasta('./data_Kbhb/train_15/Kbhb_pos_15_train.txt'))
train_neg_seqs = np.array(read_fasta('./data_Kbhb/train_15/Kbhb_neg_15_train.txt'))

train_seqs = np.concatenate( (train_pos_seqs, train_neg_seqs), axis=0 )

train = np.array(protein_to_Kmer(train_seqs,1)).astype(np.float32)

train_label = np.array( [1] * 2866 + [0] * 2866 ).astype( np.float32 )
train_label = to_categorical( train_label, num_classes=2 )

# Read the testing set
test_pos_seqs = np.array(read_fasta('./data_Kbhb/train_15/Kbhb_pos_15_test.txt'))
test_neg_seqs = np.array(read_fasta('./data_Kbhb/train_15/Kbhb_neg_15_test.txt'))

test_seqs = np.concatenate((test_pos_seqs, test_neg_seqs), axis=0)

test = np.array(protein_to_Kmer(test_seqs,1)).astype(np.float32)

test_label = np.array([1] * 318 + [0] * 318).astype(np.float32)
test_label = to_categorical(test_label, num_classes=2)

In [6]:
# Performance evaluation
def show_performance(y_true, y_pred):

    TP, FP, FN, TN = 0, 0, 0, 0

    for i in range(len(y_true)):
        if y_true[i] == 1:
            if y_pred[i] > 0.5:
                TP += 1
            else:
                FN += 1
        if y_true[i] == 0:
            if y_pred[i] > 0.5:
                FP += 1
            else:
                TN += 1


    Sn = TP / (TP + FN + 1e-06)

    Sp = TN / (FP + TN + 1e-06)

    Acc = (TP + TN) / len(y_true)

    MCC = ((TP * TN) - (FP * FN)) / np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN) + 1e-06)

    return Sn, Sp, Acc, MCC

def performance_mean(performance):
    print('Sn = %.4f ± %.4f' % (np.mean(performance[:, 0]), np.std(performance[:, 0])))
    print('Sp = %.4f ± %.4f' % (np.mean(performance[:, 1]), np.std(performance[:, 1])))
    print('Acc = %.4f ± %.4f' % (np.mean(performance[:, 2]), np.std(performance[:, 2])))
    print('Mcc = %.4f ± %.4f' % (np.mean(performance[:, 3]), np.std(performance[:, 3])))
    print('Auc = %.4f ± %.4f' % (np.mean(performance[:, 4]), np.std(performance[:, 4])))

In [7]:
# Building the model
def build_model(windows=15,embed_dim=64, weight_decay=1e-4,num_heads=8,ff_dim=64):

    print('*' * 30 + 'CNN' + '*' * 30)
    input_1 = Input(shape=(windows,))
    # Word embedding coding
    embedding = Embedding(21, embed_dim, input_length=15)
    x_1 = embedding(input_1)
    print(x_1.shape)
    #transformer = TransformerBlock(embed_dim, num_heads, ff_dim)(x_1)
    #print(transformer.shape)

    x_1 = Conv1D(filters = 32, kernel_size = 1, padding = 'same', activation= 'relu')(x_1)
    x_1 = MaxPooling1D(pool_size=2,strides=1,padding="SAME")(x_1)
    x_1 = Conv1D(filters = 32, kernel_size =  1, padding = 'same', activation= 'relu')(x_1)
    x_1 = MaxPooling1D(pool_size=2,strides=1,padding="SAME")(x_1)
    print(x_1.shape)
    
    x = Flatten()(x_1)
    x = Dense(units=2, activation='softmax')(x)


    inputs = [input_1]
    outputs = [x]

    model = Model(inputs=inputs, outputs=outputs, name="Kbhb")

    optimizer = Adam(learning_rate=1e-4, epsilon=1e-7)

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [11]:
np.random.seed(0)
tf.random.set_seed(1)  # for reproducibility
# reading model

model = build_model()


BATCH_SIZE = 32
EPOCHS = 200

# # Cross-validation
n = 10
k_fold = KFold(n_splits=n, shuffle=True, random_state=42)

all_performance = []
tprs = []
mean_fpr = np.linspace(0, 1, 100)
for fold_count, (train_index, val_index) in enumerate(k_fold.split(train)):
    print('*' * 30 + ' the ' + str(fold_count + 1) + ' fold ' + '*' * 30)
    trains, val = train[train_index], train[val_index]
    trains_label, val_label = train_label[train_index], train_label[val_index]
    model.fit(x=trains, y=trains_label, validation_data=(val, val_label), epochs=EPOCHS,
                batch_size=BATCH_SIZE, shuffle=True,
                callbacks=[EarlyStopping(monitor='val_loss', patience=13, mode='auto')],
                verbose=1)
     # 保存模型

    model.save('./models_duibi_CNN/model_fold' + str(fold_count+1) + '.h5')

    del model

    model = load_model('./models_duibi_CNN/model_fold' + str(fold_count+1) + '.h5')

    val_pred = model.predict(val, verbose=1)

    # Sn, Sp, Acc, MCC, AUC
    Sn, Sp, Acc, MCC = show_performance(val_label[:, 1], val_pred[:, 1])
    AUC = roc_auc_score(val_label[:, 1], val_pred[:, 1])
    print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

    performance = [Sn, Sp, Acc, MCC, AUC]
    all_performance.append(performance)
    
all_performance = np.array(all_performance)
print('10 fold result:', all_performance)
performance_mean = performance_mean(all_performance)

model.fit(x=train, y=train_label, validation_data=(test, test_label), epochs=EPOCHS,
                      batch_size=BATCH_SIZE, shuffle=True,
                      callbacks=[EarlyStopping(monitor='val_loss', patience=20, mode='auto')],
                      verbose=1)
model.save('./models_duibi_CNN/model_test.h5')

del model

model = load_model('./models_duibi_CNN/model_test.h5')

test_score = model.predict(test)


# Sn, Sp, Acc, MCC, AUC
Sn, Sp, Acc, MCC = show_performance(test_label[:,1], test_score[:,1])
AUC = roc_auc_score(test_label[:,1], test_score[:,1])

print('-----------------------------------------------test---------------------------------------')
print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

******************************CNN******************************
(None, 15, 64)
(None, 15, 32)
****************************** the 1 fold ******************************
Epoch 1/200
162/162 [==============================] - 5s 22ms/step - loss: 0.6909 - accuracy: 0.5417 - val_loss: 0.6894 - val_accuracy: 0.5470
Epoch 2/200
162/162 [==============================] - 3s 20ms/step - loss: 0.6826 - accuracy: 0.5727 - val_loss: 0.6785 - val_accuracy: 0.6289
Epoch 3/200
162/162 [==============================] - 3s 19ms/step - loss: 0.6659 - accuracy: 0.6458 - val_loss: 0.6608 - val_accuracy: 0.6202
Epoch 4/200
162/162 [==============================] - 3s 20ms/step - loss: 0.6408 - accuracy: 0.6582 - val_loss: 0.6345 - val_accuracy: 0.6516
Epoch 5/200
162/162 [==============================] - 3s 19ms/step - loss: 0.6187 - accuracy: 0.6687 - val_loss: 0.6216 - val_accuracy: 0.6551
Epoch 6/200
162/162 [==============================] - 3s 19ms/step - loss: 0.6059 - accuracy: 0.6764 - val_loss:

2024-06-01 20:12:35.774044: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



162/162 [==============================] - 3s 21ms/step - loss: 0.5719 - accuracy: 0.7022 - val_loss: 0.5985 - val_accuracy: 0.6951
Epoch 19/200
162/162 [==============================] - 3s 19ms/step - loss: 0.5700 - accuracy: 0.7024 - val_loss: 0.5931 - val_accuracy: 0.6986
Epoch 20/200
162/162 [==============================] - 3s 19ms/step - loss: 0.5687 - accuracy: 0.7010 - val_loss: 0.5972 - val_accuracy: 0.6969
Epoch 21/200
162/162 [==============================] - 3s 20ms/step - loss: 0.5670 - accuracy: 0.7051 - val_loss: 0.5948 - val_accuracy: 0.6986
Epoch 22/200
162/162 [==============================] - 3s 20ms/step - loss: 0.5659 - accuracy: 0.7055 - val_loss: 0.5926 - val_accuracy: 0.7003
Epoch 23/200
162/162 [==============================] - 3s 19ms/step - loss: 0.5650 - accuracy: 0.7078 - val_loss: 0.5944 - val_accuracy: 0.6969
Epoch 24/200
162/162 [==============================] - 3s 16ms/step - loss: 0.5632 - accuracy: 0.7076 - val_loss: 0.5906 - val_accuracy: 0.707

2024-06-01 20:13:25.750538: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



162/162 [==============================] - 4s 23ms/step - loss: 0.5527 - accuracy: 0.7129 - val_loss: 0.5858 - val_accuracy: 0.7143
Epoch 35/200
162/162 [==============================] - 3s 21ms/step - loss: 0.5517 - accuracy: 0.7166 - val_loss: 0.5832 - val_accuracy: 0.7091
Epoch 36/200
162/162 [==============================] - 3s 21ms/step - loss: 0.5510 - accuracy: 0.7160 - val_loss: 0.5845 - val_accuracy: 0.7108
Epoch 37/200
162/162 [==============================] - 3s 22ms/step - loss: 0.5495 - accuracy: 0.7175 - val_loss: 0.5850 - val_accuracy: 0.7160
Epoch 38/200
162/162 [==============================] - 3s 22ms/step - loss: 0.5493 - accuracy: 0.7199 - val_loss: 0.5858 - val_accuracy: 0.7178
Epoch 39/200
162/162 [==============================] - 3s 21ms/step - loss: 0.5484 - accuracy: 0.7187 - val_loss: 0.5849 - val_accuracy: 0.7143
Epoch 40/200
162/162 [==============================] - 3s 22ms/step - loss: 0.5473 - accuracy: 0.7200 - val_loss: 0.5817 - val_accuracy: 0.710

2024-06-01 20:22:54.377699: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



180/180 [==============================] - 4s 19ms/step - loss: 0.4898 - accuracy: 0.7645 - val_loss: 0.5924 - val_accuracy: 0.6934
Epoch 3/200
180/180 [==============================] - 4s 20ms/step - loss: 0.4890 - accuracy: 0.7699 - val_loss: 0.5951 - val_accuracy: 0.6840
Epoch 4/200
180/180 [==============================] - 4s 22ms/step - loss: 0.4894 - accuracy: 0.7680 - val_loss: 0.5920 - val_accuracy: 0.6918
Epoch 5/200
180/180 [==============================] - 4s 21ms/step - loss: 0.4892 - accuracy: 0.7643 - val_loss: 0.5920 - val_accuracy: 0.6950
Epoch 6/200
180/180 [==============================] - 4s 21ms/step - loss: 0.4887 - accuracy: 0.7681 - val_loss: 0.5941 - val_accuracy: 0.6840
Epoch 7/200
180/180 [==============================] - 4s 21ms/step - loss: 0.4885 - accuracy: 0.7652 - val_loss: 0.5931 - val_accuracy: 0.6918
Epoch 8/200
180/180 [==============================] - 4s 21ms/step - loss: 0.4884 - accuracy: 0.7647 - val_loss: 0.5939 - val_accuracy: 0.6824
Epoc

In [8]:
np.random.seed(0)
tf.random.set_seed(1)  # for reproducibility
# reading model

model = build_model()


BATCH_SIZE = 32
EPOCHS = 200

# # Cross-validation
n = 10
k_fold = KFold(n_splits=n, shuffle=True, random_state=42)

all_performance = []
tprs = []
mean_fpr = np.linspace(0, 1, 100)
for fold_count, (train_index, val_index) in enumerate(k_fold.split(train)):
    print('*' * 30 + ' the ' + str(fold_count + 1) + ' fold ' + '*' * 30)
    trains, val = train[train_index], train[val_index]
    trains_label, val_label = train_label[train_index], train_label[val_index]
    model.fit(x=trains, y=trains_label, validation_data=(val, val_label), epochs=EPOCHS,
                batch_size=BATCH_SIZE, shuffle=True,
                callbacks=[EarlyStopping(monitor='val_loss', patience=13, mode='auto')],
                verbose=1)
     # 保存模型

    model.save('./models_duibi_CNN/model_fold' + str(fold_count+1) + '.h5')

    del model

    model = load_model('./models_duibi_CNN/model_fold' + str(fold_count+1) + '.h5')

    val_pred = model.predict(val, verbose=1)

    # Sn, Sp, Acc, MCC, AUC
    Sn, Sp, Acc, MCC = show_performance(val_label[:, 1], val_pred[:, 1])
    AUC = roc_auc_score(val_label[:, 1], val_pred[:, 1])
    print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

    performance = [Sn, Sp, Acc, MCC, AUC]
    all_performance.append(performance)
    
all_performance = np.array(all_performance)
print('10 fold result:', all_performance)
performance_mean = performance_mean(all_performance)

model.fit(x=train, y=train_label, validation_data=(test, test_label), epochs=EPOCHS,
                      batch_size=BATCH_SIZE, shuffle=True,
                      callbacks=[EarlyStopping(monitor='val_loss', patience=20, mode='auto')],
                      verbose=1)
model.save('./models_duibi_CNN/model_test.h5')

del model

model = load_model('./models_duibi_CNN/model_test.h5')

test_score = model.predict(test)
pd.DataFrame(test_score).to_csv('./models_duibi_CNN/test_label_pred.csv', index=False)

# Sn, Sp, Acc, MCC, AUC
Sn, Sp, Acc, MCC = show_performance(test_label[:,1], test_score[:,1])
AUC = roc_auc_score(test_label[:,1], test_score[:,1])

print('-----------------------------------------------test---------------------------------------')
print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

******************************CNN******************************
(None, 15, 64)
(None, 15, 32)
****************************** the 1 fold ******************************


2024-06-12 10:21:42.424058: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11123 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6


Epoch 1/200


2024-06-12 10:21:44.983619: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8800
2024-06-12 10:21:48.460856: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2024-06-12 10:21:48.463997: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:231] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.6
2024-06-12 10:21:48.464029: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:234] Used ptxas at ptxas
2024-06-12 10:21:48.464110: W tensorflow/compiler/xla/stream_executor/gpu/redzone_allocator.cc:318] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2024-06-12 1

  1/162 [..............................] - ETA: 24:35 - loss: 0.6902 - accuracy: 0.5312

...

162/162 [==============================] - 3s 20ms/step - loss: 0.5697 - accuracy: 0.7032 - val_loss: 0.5885 - val_accuracy: 0.7108
Epoch 19/200
162/162 [==============================] - 3s 20ms/step - loss: 0.5675 - accuracy: 0.7049 - val_loss: 0.5830 - val_accuracy: 0.7160
Epoch 20/200
162/162 [==============================] - 3s 19ms/step - loss: 0.5662 - accuracy: 0.7040 - val_loss: 0.5870 - val_accuracy: 0.7125
Epoch 21/200
162/162 [==============================] - 3s 20ms/step - loss: 0.5645 - accuracy: 0.7047 - val_loss: 0.5845 - val_accuracy: 0.7056
Epoch 22/200
162/162 [==============================] - 3s 19ms/step - loss: 0.5632 - accuracy: 0.7055 - val_loss: 0.5818 - val_accuracy: 0.7125
Epoch 23/200
162/162 [==============================] - 3s 20ms/step - loss: 0.5623 - accuracy: 0.7100 - val_loss: 0.5840 - val_accuracy: 0.7056
Epoch 24/200
162/162 [==============================] - 3s 20ms/step - loss: 0.5605 - accuracy: 0.7098 - val_loss: 0.5804 - val_accuracy: 0.716

2024-06-12 10:23:41.258136: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



162/162 [==============================] - 3s 20ms/step - loss: 0.5499 - accuracy: 0.7228 - val_loss: 0.5744 - val_accuracy: 0.7160
Epoch 35/200
162/162 [==============================] - 3s 20ms/step - loss: 0.5487 - accuracy: 0.7228 - val_loss: 0.5712 - val_accuracy: 0.7213
Epoch 36/200
162/162 [==============================] - 3s 19ms/step - loss: 0.5481 - accuracy: 0.7228 - val_loss: 0.5726 - val_accuracy: 0.7143
Epoch 37/200
162/162 [==============================] - 3s 20ms/step - loss: 0.5467 - accuracy: 0.7247 - val_loss: 0.5738 - val_accuracy: 0.7230
Epoch 38/200
162/162 [==============================] - 3s 19ms/step - loss: 0.5465 - accuracy: 0.7233 - val_loss: 0.5749 - val_accuracy: 0.7230
Epoch 39/200
162/162 [==============================] - 3s 20ms/step - loss: 0.5457 - accuracy: 0.7247 - val_loss: 0.5727 - val_accuracy: 0.7213
Epoch 40/200
162/162 [==============================] - 3s 19ms/step - loss: 0.5445 - accuracy: 0.7237 - val_loss: 0.5688 - val_accuracy: 0.724

2024-06-12 10:32:34.217034: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



180/180 [==============================] - 4s 20ms/step - loss: 0.4966 - accuracy: 0.7589 - val_loss: 0.5841 - val_accuracy: 0.6792
Epoch 3/200
180/180 [==============================] - 4s 21ms/step - loss: 0.4956 - accuracy: 0.7610 - val_loss: 0.5855 - val_accuracy: 0.6840
Epoch 4/200
180/180 [==============================] - 4s 20ms/step - loss: 0.4961 - accuracy: 0.7643 - val_loss: 0.5830 - val_accuracy: 0.6808
Epoch 5/200
180/180 [==============================] - 4s 20ms/step - loss: 0.4964 - accuracy: 0.7606 - val_loss: 0.5830 - val_accuracy: 0.6777
Epoch 6/200
180/180 [==============================] - 4s 20ms/step - loss: 0.4958 - accuracy: 0.7592 - val_loss: 0.5861 - val_accuracy: 0.6871
Epoch 7/200
180/180 [==============================] - 4s 20ms/step - loss: 0.4956 - accuracy: 0.7608 - val_loss: 0.5844 - val_accuracy: 0.6855
Epoch 8/200
180/180 [==============================] - 4s 20ms/step - loss: 0.4953 - accuracy: 0.7622 - val_loss: 0.5844 - val_accuracy: 0.6855
Epoc

In [10]:
df = pd.DataFrame({
    'true_label': test_label[:,1],
    'predicted_probability': test_score[:, 1]  # 提取正类的预测概率
})
df.to_csv('./models_duibi_CNN/predictions_with_probabilities_cnn.csv', index=False)

In [17]:
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score

In [18]:
# 计算ROC曲线的TPR和FPR
fpr, tpr, thresholds = roc_curve(val_label[:, 1], val_pred[:, 1])
roc_auc = auc(fpr, tpr)

In [20]:
# 计算PR曲线的precision和recall
precision, recall, _ = precision_recall_curve(val_label[:, 1], val_pred[:, 1])
pr_auc = average_precision_score(val_label[:, 1], val_pred[:, 1])

In [22]:
roc_data = []
pr_data = []

In [26]:
# 保存ROC和PR数据
roc_data.append({'fpr': fpr, 'tpr': tpr, 'auc': roc_auc})
pr_data.append({'precision': precision, 'recall': recall, 'auc': pr_auc})
    
# 保存预测分数到CSV文件（可选，每折一个文件）
df_scores = pd.DataFrame({
    'True_Label': val_label[:, 1],
    'Prediction_Score':  val_pred[:, 1]
})
df_scores.to_csv(f'./models_duibi_CNN/prediction_scores_fold_{fold_count+1}.csv', index=False)

In [ ]:
# 计算平均ROC和PR曲线
mean_tpr = np.mean([data['tpr'] for data in roc_data], axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(np.mean([data['fpr'] for data in roc_data], axis=0), mean_tpr)
std_auc = np.std([data['auc'] for data in roc_data])

mean_precision = np.mean([data['precision'] for data in pr_data], axis=0)
mean_recall = np.mean([data['recall'] for data in pr_data], axis=0)
mean_ap = np.mean([data['auc'] for data in pr_data])
std_ap = np.std([data['auc'] for data in pr_data])